In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#Converting doc_mag.txt to a list of tuples (did,maxf,magnitudesq)
DocMag_text=open( "/content/drive/My Drive/doc_mag.txt","r")
DocMag_lines=DocMag_text.readlines()
DocMag_lines = list(map(eval, DocMag_lines))
DocMag_lines = list(map(lambda x : (x[0],x[1][0],x[1][1]),DocMag_lines))
print(DocMag_lines[0])

(16474, 7, 0.08380229270896267)


In [4]:
#Converting inv_idx.txt to a list of tuples (word,[(did,freq,tfidf),...)
posting_text=open( "/content/drive/My Drive/inv_idx.txt","r")
posting_lines = posting_text.readlines()
posting_lines = list(map(eval, posting_lines));
posting_lines = list( map(lambda x: (x[0], str(x[1])), posting_lines) )
print(posting_lines[0])

('apr', '[(16474, 1, 3.011322572874006e-05), (14751, 1, 0.00010539629005059021), (12470, 1, 7.026419336706015e-05), (16897, 1, 7.026419336706015e-05), (13243, 1, 4.215851602023609e-05), (12325, 1, 4.215851602023609e-05), (16123, 1, 2.6349072512647553e-05), (16652, 1, 7.026419336706015e-05), (15269, 1, 5.269814502529511e-05), (16804, 1, 2.1079258010118044e-05), (14841, 1, 0.00010539629005059021), (12592, 1, 7.026419336706015e-05), (16379, 1, 7.026419336706015e-05), (16655, 1, 3.5132096683530073e-05), (15741, 1, 5.269814502529511e-05), (12383, 1, 2.3421397789020048e-05), (11861, 1, 4.215851602023609e-05), (17340, 1, 5.269814502529511e-05), (16585, 1, 3.5132096683530073e-05), (17007, 1, 1.1710698894510024e-05), (14920, 1, 7.026419336706015e-05), (15675, 1, 5.269814502529511e-05), (11896, 1, 2.1079258010118044e-05), (15529, 1, 4.215851602023609e-05), (14906, 1, 3.011322572874006e-05), (15969, 1, 3.5132096683530073e-05), (15552, 1, 2.1079258010118044e-05), (13211, 1, 7.026419336706015e-05),

In [5]:
#Creating two data base tables and inserting the values from the two text files into these tables
import sqlite3
conn1 = sqlite3.connect('DocMag.db')
c1 = conn1.cursor()
c1.execute("""CREATE TABLE IF NOT EXISTS DocMag (
             docid integer PRIMARY KEY,
             maxf integer,
             mag float)
         """)
#Creating Index using B-TREE on docid coloumn of DocMag table
c1.execute("""CREATE UNIQUE INDEX idx_DocMag_docid ON DocMag (docid)""")
c1.executemany("INSERT INTO DocMag VAlUES (?,?,?)", DocMag_lines)
conn1.commit()
c1.execute("SELECT * FROM DocMag")
print(c1.fetchone())

conn = sqlite3.connect('Posting.db')
c = conn.cursor()
c.execute("""CREATE TABLE IF NOT EXISTS Posting (
             word VARCHAR(100) PRIMARY KEY,
             postinglist_freq_tfidf text)
         """)
#Creating Index using B-TREE on word coloumn of Posting table
c.execute("""CREATE UNIQUE INDEX idx_Posting_word ON Posting (word)""")
c.executemany("INSERT INTO Posting VAlUES (?,?)", posting_lines)
conn.commit()
c.execute("SELECT * FROM Posting")
print(c.fetchone())

(1, 14, 0.1431944602432088)
('apr', '[(16474, 1, 3.011322572874006e-05), (14751, 1, 0.00010539629005059021), (12470, 1, 7.026419336706015e-05), (16897, 1, 7.026419336706015e-05), (13243, 1, 4.215851602023609e-05), (12325, 1, 4.215851602023609e-05), (16123, 1, 2.6349072512647553e-05), (16652, 1, 7.026419336706015e-05), (15269, 1, 5.269814502529511e-05), (16804, 1, 2.1079258010118044e-05), (14841, 1, 0.00010539629005059021), (12592, 1, 7.026419336706015e-05), (16379, 1, 7.026419336706015e-05), (16655, 1, 3.5132096683530073e-05), (15741, 1, 5.269814502529511e-05), (12383, 1, 2.3421397789020048e-05), (11861, 1, 4.215851602023609e-05), (17340, 1, 5.269814502529511e-05), (16585, 1, 3.5132096683530073e-05), (17007, 1, 1.1710698894510024e-05), (14920, 1, 7.026419336706015e-05), (15675, 1, 5.269814502529511e-05), (11896, 1, 2.1079258010118044e-05), (15529, 1, 4.215851602023609e-05), (14906, 1, 3.011322572874006e-05), (15969, 1, 3.5132096683530073e-05), (15552, 1, 2.1079258010118044e-05), (13211

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
from bs4 import BeautifulSoup
from collections import Counter
import re
from nltk.corpus import stopwords
import math
stop_words = set(stopwords.words('english'))
# for a given query returning a list of tuples of the form (w,freq/maxfreq)
def dw(query):
  words = re.findall('[a-zA-Z]+', query)
  words = [w.lower() for w in words]
  words = [w for w in words if not w in stop_words]
  counter = Counter(words)
  most_common,maxf = counter.most_common(1)[0]
  return [(w,counter[w]/maxf) for w in counter]
# Inputing the query
query = input("Enter Query: ") 
word_TF = dw(query)
dictOfDocuments={}
magOfQuery=0
# Calculating Cosine silimarity of each document with the query
for i in range(len(word_TF)):
  c.execute("SELECT postinglist_freq_tfidf FROM Posting where word='"+word_TF[i][0]+"'")
  text=c.fetchone()
  if text!=None:
    postinglist_freq_tfidf=eval(text[0])
    tfIdf=word_TF[i][1]/len(postinglist_freq_tfidf)
    magOfQuery=magOfQuery+(tfIdf*tfIdf)
    for j in range(len(postinglist_freq_tfidf)):
      if postinglist_freq_tfidf[j][0] in dictOfDocuments:
        dictOfDocuments[postinglist_freq_tfidf[j][0]] = tfIdf*postinglist_freq_tfidf[j][2]+ dictOfDocuments[postinglist_freq_tfidf[j][0]]
      else:
        dictOfDocuments[postinglist_freq_tfidf[j][0]]= tfIdf*postinglist_freq_tfidf[j][2]
for docId in dictOfDocuments:
  c1.execute("SELECT mag FROM DocMag where docid= ?",(docId,))
  dictOfDocuments[docId]=dictOfDocuments[docId]/(math.sqrt(c1.fetchone()[0])*math.sqrt(magOfQuery))
# Sorting and printing the documents in a reverse order acording to Cosine similarity with the query
sortedListofTuples = sorted(dictOfDocuments.items() , reverse=True, key=lambda x: x[1])
print('Search results(documents are sorted according to Cosine similarity with query):')
if len(sortedListofTuples)==0:
  print('There is no matching document for this query.')
else:
  print('did     :', '  Cosine Similarity')
row = "{did:<7s} :   {cosine}".format
for elem in sortedListofTuples :
   print(row(did='d'+str(elem[0]), cosine=elem[1]))

Enter Query: vieille and lamy or format
Search results(documents are sorted according to Cosine similarity with query):
did     :   Cosine Similarity
d6075   :   0.4028701753790083
d14875  :   0.19813292694977744
d14876  :   0.1352635377573943
d18462  :   0.020890838799602362
d1831   :   0.009768329168919666
d1822   :   0.008356301970356332
d12582  :   0.007691824737695096
d13112  :   0.006666656885798142
d9403   :   0.005735375006740295
d6376   :   0.005320259150898364
d2881   :   0.004135150390035177
d18368  :   0.0031926207822173553
d17928  :   0.0024241735712224118
d9150   :   0.0018895301917045478
d18292  :   0.0017992443665639634
